### Fonctions utilitaires

Compléter en ajoutant les fonctions de padding et de découpage en bloc.

In [1]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)


def psnr(original, compressed):
    mse = np.mean((original.astype(int) - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


### Utilitaires pour le calcul des palettes et des patchs

Fonctions auxiliaires pour manipuler les palettes, les pixels et les blocs. 

In [2]:
def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)

def save(matPix, filename):
    Image.fromarray(matPix).save(filename)

def padding(filename):
    mat = load(filename)

    ligne_mat = mat.shape[0]
    colonne_mat = mat.shape[1]

    #Obtenir le nombre de ligne et de colonne à rajouter.
    ligne_mat_manquante = (4-ligne_mat % 4) % 4  
    colonne_mat_manquante = (4-colonne_mat % 4) % 4

    #Création d'une matrice vide de la taille souhaitée. 
    mat_padding = np.zeros((ligne_mat+ligne_mat_manquante,colonne_mat+colonne_mat_manquante,3),dtype = mat.dtype)
    mat_padding[:ligne_mat, :colonne_mat] = mat

    return mat_padding 

def decoupe_matrice(mat):
    ligne_mat = mat.shape[0]
    colonne_mat = mat.shape[1]

    #Création d'une liste qui va contenir tout les carre de 4x4 pixel en RGB.
    global L
    L=[]
    for i in range(0, ligne_mat, 4):
        for j in range(0, colonne_mat, 4):
            carre = mat[i:i+4, j:j+4]
            L.append(carre)

    return L

def non_padding(filename, ligne_de_base, colonne_de_base):
    #Padding
    mat_padding = padding(filename)
    #Découpe de la matrice
    L = decoupe_matrice(mat_padding)

    #Dimensions de la matrice recollée.
    ligne_mat = mat_padding.shape[0]
    colonne_mat = mat_padding.shape[1]

    #Création de la matrice recollée.
    mat_recoller = np.zeros((ligne_mat, colonne_mat, 3), dtype=mat_padding.dtype)
    for i in range(len(L)):
        ligne_idx = (i // (colonne_de_base // 4)) * 4
        col_idx = (i % (colonne_de_base // 4)) * 4
        mat_recoller[ligne_idx:ligne_idx+4, col_idx:col_idx+4] = L[i]

    #Retour à la matrice initiale.
    mat_de_base = mat_recoller[:ligne_de_base,:colonne_de_base]
    
    return mat_de_base

mat_padding = padding("proc.jpg")
save(mat_padding, "proc_padding.jpg")
mat_de_base = non_padding("proc.jpg",1537,2048)
save(mat_de_base, "proc_non_padding.jpg")

In [3]:
def tronque(n, p):
    n_binaire = []
    while n != 0:
        n_binaire.append(n % 2)
        n = n // 2
    n_binaire.reverse()

    n_binaire = n_binaire[:-p]

    n_decimal = 0
    for i in range(len(n_binaire)):
        n_decimal += n_binaire[i] * (2 ** (len(n_binaire) - 1 - i))

    return n_decimal

n = 255
p = 3   
n_tronque = tronque(n, p)
print("Nouvelle valeur", n_tronque)



def palette(a, b):
    a = [tronque(a[0],3), tronque(a[1],2), tronque(a[2],3)]
    b = [tronque(b[0],3), tronque(b[1],2), tronque(b[2],3)]
    
    palette = np.zeros((4, 3), dtype=int)
    palette[0] = a
    palette[1] = np.round(2 * np.array(a) / 3 + np.array(b) / 3)
    palette[2] = np.round(np.array(a) / 3 + 2 * np.array(b) / 3)
    palette[3] = b
    
    return palette

a = [90,90,90]
b = [180,180,180]
c = palette(a, b)
print(c)



def couleur_la_plus_proche(palette, pixel):
    indice_couleur_plus_proche = 0
    distance_euclidienne_la_plus_petite = np.linalg.norm(palette[0].astype(int) - pixel)
    for i in range (len(palette)):
        distance_euclidienne = np.linalg.norm(palette[i].astype(int) - pixel)
        if distance_euclidienne < distance_euclidienne_la_plus_petite:
            indice_couleur_plus_proche = i

    return indice_couleur_plus_proche

pixel = [100, 100, 100]
print(couleur_la_plus_proche(c, pixel))



def patch(tableaux_des_indices, palette):
    patch = ""
    for indices in tableaux_des_indices:
        pixel = L[0][indices[0], indices[1]]  #Pixel du premier block de 4x4 pixel
        indice_couleur_plus_proche = couleur_la_plus_proche(palette, pixel)
        indice_couleur_plus_proche_binaire = bin(indice_couleur_plus_proche)  #Binaire
        patch += str(indice_couleur_plus_proche_binaire)

    return patch

tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
indices_patchés = patch(tableaux_des_indices, c)
print(indices_patchés)

Nouvelle valeur 31
[[11 22 11]
 [15 30 15]
 [18 37 18]
 [22 45 22]]
3
0b110b110b110b110b110b110b110b110b110b110b110b110b110b110b110b11


### Les méthodes de choix des couleurs de la palette


On peut essayer plusieurs stratégies pour calculer les deux couleurs a et b:
* Calculer la couleur minimale et maximale
* Prendre des couleurs moyennes (calculer moyenne et variance pour trouver les meilleurs valeurs) 
* Calculer 10 valeurs au hasard et prendre la meilleure
* Partir d'un choix et perturber pour améliorer le résultat: on teste la meilleure modification de 16 d'un des canaux d'une des couleurs

### Encodage d'un bloc

Écrire ici le code qui permet d'encoder un bloc et une palette en un entier.

### Écriture et lecture dans un fichier

### Fonctions de test

Écrire ici tous vos tests. Chaque fonction écrite doit être testée.